In [3]:
import tensorflow as tf

prototypes = 200


class DS1(tf.keras.layers.Layer):
    def __init__(self, units, input_dim):
        super(DS1, self).__init__()
        self.w = self.add_weight(
            name='Prototypes',
            shape=(units, input_dim),
            initializer='random_normal',
            trainable=True
        )
        self.units = units

    def call(self, inputs):
        for i in range(self.units):
            if i == 0:
                un_mass_i = tf.subtract(self.w[i, :], inputs, name=None)
                un_mass_i = tf.square(un_mass_i, name=None)
                un_mass_i = tf.reduce_sum(un_mass_i, -1, keepdims=True)
                un_mass = un_mass_i

            if i >= 1:
                un_mass_i = tf.subtract(self.w[i, :], inputs, name=None)
                un_mass_i = tf.square(un_mass_i, name=None)  #(N, 128)
                un_mass_i = tf.reduce_sum(un_mass_i, -1, keepdims=True)  #(N, 1)
                un_mass = tf.concat([un_mass, un_mass_i], -1)  #(N, 200, 1)

        return un_mass

In [4]:
a = tf.keras.layers.Input(128)
print("a = " + str(a))
out1 = DS1(prototypes, 128)(a)
print("out1 =" + str(out1))

a = Tensor("input_2:0", shape=(None, 128), dtype=float32)
out1 =Tensor("d_s1_1/Identity:0", shape=(None, 200), dtype=float32)


In [5]:
class DS1_activate(tf.keras.layers.Layer):
    def __init__(self, input_dim):
        super(DS1_activate, self).__init__()
        self.xi = self.add_weight(
            name='xi',
            shape=(1, input_dim),
            initializer='random_normal',
            trainable=True
        )
        self.eta = self.add_weight(
            name='eta',
            shape=(1, input_dim),
            initializer='random_normal',
            trainable=True
        )
        self.input_dim = input_dim

    def call(self, inputs):
        gamma = tf.square(self.eta, name=None)
        alpha = tf.negative(self.xi, name=None)
        alpha = tf.exp(alpha, name=None) + 1
        alpha = tf.divide(1, alpha, name=None)
        si = tf.multiply(gamma, inputs, name=None)
        si = tf.negative(si, name=None)
        si = tf.exp(si, name=None)
        si = tf.multiply(si, alpha, name=None)
        si = tf.divide(si, (tf.reduce_max(si, axis=-1, keepdims=True) + 0.001), name=None)

        return si

In [6]:
out2 = DS1_activate(prototypes)(out1)
print("out2 =" + str(out2))

out2 =Tensor("d_s1_activate/Identity:0", shape=(None, 200), dtype=float32)


In [7]:
class DS2(tf.keras.layers.Layer):
    def __init__(self, input_dim, num_class):
        super(DS2, self).__init__()
        self.beta = self.add_weight(
            name='beta',
            shape=(input_dim, num_class),
            initializer='random_normal',
            trainable=True
        )
        self.input_dim = input_dim
        self.num_class = num_class

    def call(self, inputs):
        beta = tf.square(self.beta, name=None)  #(200,10)
        beta_sum = tf.reduce_sum(beta, -1, keepdims=True)  #(200, 1)
        u = tf.divide(beta, beta_sum, name=None)  #(200, 10)
        inputs_new = tf.expand_dims(inputs, -1)  #(None,200) -> (None,200, 1)
        for i in range(self.input_dim):
            if i == 0:
                # (1, 10) * (None,1,1) -> (None,10)
                mass_prototype_i = tf.multiply(u[i, :], inputs_new[:, i], name=None)
                mass_prototype = tf.expand_dims(mass_prototype_i, -2)  # (None,1,10)

            if i >= 1:
                mass_prototype_i = tf.expand_dims(tf.multiply(u[i, :], inputs_new[:, i], name=None), -2)
                mass_prototype = tf.concat([mass_prototype, mass_prototype_i], -2)  # (None, 200, 10)

        mass_prototype = tf.convert_to_tensor(mass_prototype)
        return mass_prototype

In [8]:
out3 = DS2(prototypes, 10)(out2)
print("out3 = " + str(out3))

out3 = Tensor("d_s2/Identity:0", shape=(None, 200, 10), dtype=float32)


In [9]:
class DS2_omega(tf.keras.layers.Layer):
    def __init__(self, input_dim, num_class):
        super(DS2_omega, self).__init__()
        self.input_dim = input_dim
        self.num_class = num_class

    def call(self, inputs):
        mass_omega_sum = tf.reduce_sum(inputs, -1, keepdims=True)  # (None, 200, 1)
        mass_omega_sum = tf.subtract(1., mass_omega_sum[:, :, 0], name=None)  # (None, 200)
        mass_omega_sum = tf.expand_dims(mass_omega_sum, -1)  # (None, 200, 1)
        mass_with_omega = tf.concat([inputs, mass_omega_sum], -1)  # (None, 200, 11)
        return mass_with_omega

In [10]:
out4 = DS2_omega(prototypes, 10)(out3)
print("out4 = " + str(out4))

out4 = Tensor("d_s2_omega/Identity:0", shape=(None, 200, 11), dtype=float32)


In [11]:
class DS3_Dempster(tf.keras.layers.Layer):
    def __init__(self, input_dim, num_class):
        super(DS3_Dempster, self).__init__()
        self.input_dim = input_dim
        self.num_class = num_class

    def call(self, inputs):
        m1 = inputs[:, 0, :]  # (None, 11)
        omega1 = tf.expand_dims(inputs[:, 0, -1], -1)  # (None, 1 )
        for i in range(self.input_dim - 1):
            m2 = inputs[:, (i + 1), :]
            omega2 = tf.expand_dims(inputs[:, (i + 1), -1], -1)  # (None, 1)
            combine1 = tf.multiply(m1, m2, name=None)  # (None, 11)
            combine2 = tf.multiply(m1, omega2, name=None)  # (Nobe, 11)
            combine3 = tf.multiply(omega1, m2, name=None)
            combine1_2 = tf.add(combine1, combine2, name=None)
            combine2_3 = tf.add(combine1_2, combine3, name=None)
            combine2_3 = combine2_3 / tf.reduce_sum(combine2_3, axis=-1, keepdims=True)
            m1 = combine2_3
            omega1 = tf.expand_dims(combine2_3[:, -1], -1)
        return m1

In [12]:
out5 = DS3_Dempster(prototypes, 10)(out4)
print("out5 = " + str(out5))

out5 = Tensor("d_s3__dempster/Identity:0", shape=(None, 11), dtype=float32)


In [13]:
class DS3_normalize(tf.keras.layers.Layer):
    def __init__(self):
        super(DS3_normalize, self).__init__()

    def call(self, inputs):
        mass_combine_normalize = inputs / tf.reduce_sum(inputs, axis=-1, keepdims=True)
        return mass_combine_normalize

In [14]:
out6 = DS3_normalize()(out5)
print("out6 = " + str(out6))

out6 = Tensor("d_s3_normalize/Identity:0", shape=(None, 11), dtype=float32)


In [15]:
class DM(tf.keras.layers.Layer):
    def __init__(self, nu, num_class):
        super(DM, self).__init__()
        self.nu = nu
        self.num_class = num_class

    def call(self, inputs):
        upper = tf.expand_dims((1-self.nu) * inputs[:,-1], -1)  # (None, 1)
        upper = tf.tile(upper, [1, self.num_class + 1])  # (None, 11)
        outputs = tf.add(inputs, upper, name=None)[:, 0:-1]  # (None, 10)
        return outputs

In [16]:
out7 = DM(0.9, 10)(out6)
print("out7 = " + str(out7))

out7 = Tensor("dm/Identity:0", shape=(None, 10), dtype=float32)
